In [1]:
# %load predict.py
from keras.preprocessing.image import img_to_array,load_img,array_to_img
from keras.models import *
from keras.layers import *
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.optimizers import Adam,SGD
import glob

class dataProcess(object):
    def __init__(self, out_rows, out_cols, data_path="./orig_data/image", label_path="./orig_data/mask",
                 test_path="./orig_data/test", npy_path="./npy_data", img_type="png"):
        # 数据处理类，初始化
        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.img_type = img_type
        self.test_path = test_path
        self.npy_path = npy_path

    # 创建训练数据
    def create_train_data(self):
        i = 0
        print('-' * 30)
        print('Creating training images...')
        print('-' * 30)
        imgs = glob.glob(self.data_path + "/*." + self.img_type)
        img_len=len(imgs)
        print(len(imgs))

		# 此处有改动，1变为3
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
        imglabels = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)
        for imgname in imgs:
            midname = imgname[imgname.rindex("\\") + 1:]
            img = load_img(self.data_path + "/" + midname, color_mode='rgb')
            label = load_img(self.label_path + "/" + midname, grayscale=True)
            img = img_to_array(img)
            label = img_to_array(label)
            # img = cv2.imread(self.data_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # label = cv2.imread(self.label_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # img = np.array([img])
            # label = np.array([label])
            imgdatas[i] = img
            imglabels[i] = label
            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, len(imgs)))
            i += 1
        print('loading done')
        np.save(self.npy_path + '/imgs_train.npy', imgdatas)
        np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')

    # 创建测试数据
    def create_test_data(self):
        i = 0
        print('-' * 30)
        print('Creating test images...')
        print('-' * 30)
        imgs = glob.glob(self.test_path + "/*." + self.img_type)
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
        for imgname in imgs:
            midname = imgname[imgname.rindex("\\") + 1:]
            img = load_img(self.test_path + "/" + midname, color_mode='rgb')
            img = img_to_array(img)
            # img = cv2.imread(self.test_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # img = np.array([img])
            imgdatas[i] = img
            i += 1
        print('loading done')
        np.save(self.npy_path + '/imgs_test.npy', imgdatas)
        print('Saving to imgs_test.npy files done.')

    # 加载训练图片与mask
    def load_train_data(self):
        print('-' * 30)
        print('load train images...')
        print('-' * 30)
        imgs_train = np.load(self.npy_path + "/imgs_train.npy")
        imgs_mask_train = np.load(self.npy_path + "/imgs_mask_train.npy")
        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        imgs_train /= 255
        # mean = imgs_train.mean(axis=0)
        # imgs_train -= mean
        imgs_mask_train /= 255
        # 做一个阈值处理，输出的概率值大于0.5的就认为是对象，否则认为是背景
        imgs_mask_train[imgs_mask_train > 0.5] = 1
        imgs_mask_train[imgs_mask_train <= 0.5] = 0
        return imgs_train, imgs_mask_train

    # 加载测试图片
    def load_test_data(self):
        print('-' * 30)
        print('load test images...')
        print('-' * 30)
        imgs_test = np.load(self.npy_path + "/imgs_test.npy")
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255
        # mean = imgs_test.mean(axis=0)
        # imgs_test -= mean
        return imgs_test

IMAGE_ORDERING = 'channels_last'
def VGG16(n_classes,importModel = None):

    image_input = Input(shape = (512,512,3))
    #block1
    x = Conv2D(64,(3,3),activation = 'relu',padding = 'same',name = 'block1_conv1')(image_input)
    x = Conv2D(64,(3,3),activation = 'relu',padding = 'same', name = 'block1_conv2')(x)
    f1 = x

    x = MaxPooling2D((2,2), strides = (2,2), name = 'block1_pool')(x)
    x = Conv2D(128,(3,3),activation = 'relu',padding = 'same',name = 'block2_conv1')(x)
    x = Conv2D(128,(3,3),activation = 'relu',padding = 'same',name = 'block2_conv2')(x)
    f2 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block2_pool')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv1')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv2')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv3')(x)
    f3 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block3_pool')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv1')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv2')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv3')(x)
    f4 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block4_pool')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv1')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv2')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv3')(x)
    f5 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block5_pool')(x)


    # x = Flatten(name='flatten')(x)
    # x = Dense(4096, activation='relu', name='fc1')(x)
    # x = Dense(4096, activation='relu+', name='fc2')(x)
    # x = Dense(1000, activation='softmax', name='predictions')(x)
    vgg = Model(image_input, x, name='vgg16')
    # vgg.load_weights("C:/Users/hq/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

    levels = [f1, f2, f3, f4, f5]

    o = f5
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f4], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f3], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f2], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f1], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o)
    o= Conv2D(1, 1, activation='sigmoid')(o)

    # conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
    # o_shape = Model(image_input, o).output_shape
	#
    # output_height = o_shape[1]
    # output_width = o_shape[2]
	#
    # o = (Reshape((n_classes, output_height * output_width)))(o)
	#
    # o = (Permute((2, 1)))(o)
    # o = (Activation('softmax'))(o)
    model = Model(image_input, o)

    # model.outputWidth = output_width
    # model.outputHeight = output_height

    return model

    #Classification block
    # x = Flatten(name = 'flatten')(x)
    # x = Dense(4096,activation = 'relu',name = 'fc1')(x)
    # x = Dense(4096,activation = 'relu',name = 'fc2')(x)
    # x = Dense(num_classes,activation = 'softmax',name = 'fc3')(x)

    # if importModel:
    #     model = Sequential()
    #     model.load_weights(importModel)
    # return model


def load_data():
    mydata = dataProcess(512, 512)
    imgs_train, imgs_mask_train = mydata.load_train_data()
    imgs_test = mydata.load_test_data()
    return imgs_train, imgs_mask_train, imgs_test

# 将预测生成的numpy数组转换为灰度图
def save_img_gray():
    print("array to image")
    imgs = np.load('./results/pred_imgs_mask_test.npy')
    for i in range(imgs.shape[0]):
        img = imgs[i]
        img = array_to_img(img)
        img.save("./results/%d.jpg" % (i))

# 将预测生成的numpy数组转换为二值图
def save_img_binary():
    print("array to image")
    imgs = np.load('./results/imgs_mask_test.npy')
    imgs[imgs > 0.5] = 1
    imgs[imgs <= 0.5] = 0
    for i in range(imgs.shape[0]):
        img = imgs[i]
        img = array_to_img(img)
        img.save("./results/%d_binary.png" % (i + 1))
if __name__ == "__main__":
    imgs_train, imgs_mask_train, imgs_test=load_data()
    model = VGG16(2,'vgg16_weights.h5')
    model.load_weights('my_VGG_unet.hdf5')
    model.compile(optimizer=Adam(lr = 1e-4), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
    np.save('./results_vggunet//pred_imgs_mask_test.npy', imgs_mask_test)
    save_img_gray()


    # model = VGG16(2, 'vgg16_weights.h5')
    # print(model.summary())


Using TensorFlow backend.
W0611 22:05:34.276548 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0611 22:05:34.301534 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0611 22:05:34.304514 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0611 22:05:34.342515 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------


W0611 22:05:34.636336 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0611 22:05:34.637324 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0611 22:05:35.001113 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0611 22:05:35.104681 11092 deprecation_wrapper.py:119] From D:\anaconda_64anzhuang\lib\site-packages\keras\backend\tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0611 22:05:36.267

2/2 [==============================] - 7s 4s/step
